In [1]:
import torch
import torch.nn as nn

inputs=torch.tensor(  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) #
class maskedattention(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_query=nn.Linear(3,3,bias=False)
        self.W_key=nn.Linear(3,3,bias=False)
        self.W_value=nn.Linear(3,3,bias=False)
        self.dropout=nn.Dropout(0.5)
        self.register_buffer('mask', torch.triu(torch.ones(6, 6), diagonal=1)) # New
    def forward(self,inputs):
        b,context_length,dim=inputs.shape
        key=self.W_key(inputs)
        value=self.W_value(inputs)
        query=self.W_query(inputs)
        attention_score=query@key.transpose(1,2)## do transpose on 2 and 3 dimension 1 is batch
        attention_score.masked_fill(self.mask.bool()[:context_length,:context_length],-torch.inf)
        attn_weights = torch.softmax(
            attention_score / key.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vector=attn_weights@value
        return context_vector
maskedAttention=maskedattention()
torch.manual_seed(123)
maskedAttention(batch)

torch.Size([2, 6, 3])


tensor([[[ 0.5745, -0.8028, -0.6604],
         [ 0.1437, -0.1911, -0.1668],
         [ 0.2736, -0.2789, -0.1469],
         [ 0.2075, -0.4222, -0.4252],
         [ 0.0622, -0.2322, -0.2596],
         [ 0.1446, -0.1923, -0.1679]],

        [[ 0.3764, -0.4007, -0.2573],
         [ 0.2725, -0.2810, -0.1529],
         [ 0.2889, -0.3803, -0.3276],
         [ 0.4218, -0.6232, -0.5261],
         [ 0.4078, -0.4637, -0.3156],
         [ 0.1859, -0.3125, -0.2394]]], grad_fn=<UnsafeViewBackward0>)